In [35]:
import pandas as pd
import json
import flair
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
import torch
import re
import numpy as np

In [678]:
cuda = torch.device('cuda')
cpu = torch.device('cpu')

In [679]:
device = cpu

In [680]:
flair.device = device

In [681]:
emb = WordEmbeddings('pl')

In [682]:
with open("recipes_pretty.json") as file:
    data = json.load(file)

In [731]:
data

[{'link': ['https://kkpolska.pl/przepis/98/tropikalny_drink_bezalkoholowy_z_guawa_i_imbirem_.html'],
  'title': ['Tropikalny drink bezalkoholowy z guawą i imbirem '],
  'photo_link': ['images/przepisy/guawa_imbir.jpg'],
  'ingredients': [['3 łyżeczki soku z limonki \r'],
   ['1/2 szklanki napoju Rubicon Deluxe Guawa \r'],
   ['2 łyżki miodu \r'],
   ['2 łyżeczki imbiru ']],
  'preparation': ['Wszystkie składniki umieścić w blenderze i zmiksować. Tropikalny drink bardzo dobrze smakuje schłodzony. ']},
 {'link': ['https://kkpolska.pl/przepis/38/tropikalne_smoothie_z_lychee.html'],
  'title': ['Tropikalne smoothie z lychee'],
  'photo_link': ['images/przepisy/smoothie_lychee.jpg'],
  'ingredients': [['300ml jogurtu naturalnego 2%\r'],
   ['10 owoców lychee\r'],
   ['2 banany\r'],
   ['200g ananasa\r'],
   ['1 szklanka kruszonego lodu\r'],
   ['miód']],
  'preparation': ['1. Banany, lychee, ananasa pokroić na mniejsze kawałki.\r\n2. Owoce zalać jogurtem, dodać kruszony lód.\r\n3. Całość zm

# Wyrazy kluczowe

In [683]:
temp = Sentence('jedzenie jednostka narzędzie')
emb.embed(temp)
jedzenie = temp[0].embedding
jednostka = temp[1].embedding
narzedzie = temp[2].embedding

# Przykładowy składnik

### Przygotowanie

In [684]:
ingredient = Sentence(data[0]["ingredients"][0][0])

In [685]:
emb.embed(ingredient)

[Sentence: "3 łyżeczki soku z limonki 
 " - 6 Tokens]

In [686]:
for i in ingredient:
    print(i, i.embedding[:5])

Token: 1 3 tensor([ 0.2346,  0.2216,  0.0498, -0.7787,  0.3949])
Token: 2 łyżeczki tensor([-0.1335,  0.2546,  0.0863, -1.0030,  0.3891])
Token: 3 soku tensor([-0.3744, -0.0344, -0.4256, -0.9503,  0.4655])
Token: 4 z tensor([-0.1382, -0.0861, -0.0652, -0.1541, -0.0471])
Token: 5 limonki tensor([-0.4339,  0.4223, -0.4476, -0.9961,  0.3207])
 oken: 6 tensor([0., 0., 0., 0., 0.])


### Odległość L2 od słowa "jedzenie"

In [687]:
for i in ingredient:
    print(torch.norm(i.embedding - jedzenie))

tensor(5.8111)
tensor(5.0761)
tensor(5.6460)
tensor(5.0353)
tensor(5.4801)
tensor(4.6423)


###  Podobieństwo cosinusowe od słowa jedzenie

In [688]:
def cos(v,u): return (v @ u)/v.norm()/u.norm()

In [689]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, jedzenie))

Token: 1 3
tensor(0.1270)
Token: 2 łyżeczki
tensor(0.4890)
Token: 3 soku
tensor(0.4162)
Token: 4 z
tensor(0.1661)
Token: 5 limonki
tensor(0.3724)


###  Podobieństwo cosinusowe od słowa jednostka

In [690]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, jednostka))

Token: 1 3
tensor(0.1691)
Token: 2 łyżeczki
tensor(0.0952)
Token: 3 soku
tensor(0.1207)
Token: 4 z
tensor(0.0962)
Token: 5 limonki
tensor(0.1066)


### Pod. do narzędzie

In [691]:
for i in ingredient:
    if i.embedding.norm() > 0.001:
        print(i)
        print(cos(i.embedding, narzedzie))

Token: 1 3
tensor(0.1972)
Token: 2 łyżeczki
tensor(0.3519)
Token: 3 soku
tensor(0.1850)
Token: 4 z
tensor(0.0672)
Token: 5 limonki
tensor(0.1857)


In [692]:
for d in data:
    for i in d['ingredients']:
        print(i[0].split())

['3', 'łyżeczki', 'soku', 'z', 'limonki']
['1/2', 'szklanki', 'napoju', 'Rubicon', 'Deluxe', 'Guawa']
['2', 'łyżki', 'miodu']
['2', 'łyżeczki', 'imbiru']
['300ml', 'jogurtu', 'naturalnego', '2%']
['10', 'owoców', 'lychee']
['2', 'banany']
['200g', 'ananasa']
['1', 'szklanka', 'kruszonego', 'lodu']
['miód']
['1', 'szklanka', 'wody', 'kokosowej']
['1', 'szklanka', 'owoców', 'mango', '(świeżych', 'lub', 'mrożonych)']
['1', 'obrany', 'banan']
['200g', 'jogurtu', 'greckiego']
['1', 'łyżka', 'cukru', 'brązowego']
['2', 'szklanki', 'puree', 'z', 'mango']
['1/2', 'szklanki', 'mleczka', 'kokosowego']
['2', 'łyżeczki', 'brązowego', 'cukru']
['1', 'szklanka', 'wody', 'mineralnej', 'niegazowanej']
['4', 'łyżki', 'stołowe', 'nasion', 'Chia']
['2', 'łyżeczki', 'soku', 'z', 'limonki']
['100ml', 'wody', 'kokosowej']
['100g', 'jogurtu', 'naturalnego']
['1/2', 'mango']
['150g', 'truskawek']
['lód']
['Napój', 'Mango', 'Rubicon', 'Deluxe']
['1/2', 'cytryny']
['6', 'mandarynek']
['125g', 'ananasa']
['1', '

['1', 'drobno', 'posiekana', 'papryczka', 'chili', '(opcjonalnie)']
[]
['Składniki', 'na', 'sos:', '•', '2', 'łyżki', 'stołowe', 'sosu', 'ostrygowego']
['2', 'łyżki', 'stołowe', 'sosu', 'po', 'pekińsku']
['2', 'łyżki', 'stołowe', 'wina', 'ryżowego']
['3', 'łyżki', 'stołowe', 'wody']
['350', 'g', 'udek', 'z', 'kurczaka']
['200', 'g', 'świeżego', 'makaronu', 'jajecznego']
['1/2', 'ogórka', 'pokrojonego', 'w', 'cienkie', 'paski']
['1', 'duża', 'marchew,', 'obrana', 'i', 'pokrojona', 'w', 'cienkie', 'paski']
[]
['•', '2', 'łyżki', 'stołowe', 'sosu', 'ostrygowego', 'Lee', 'Kum', 'Kee']
['2', 'łyżki', 'stołowe', 'sosu', 'sojowego', 'słodkiego', 'do', 'pierożków', 'Lee', 'Kum', 'Kee']
['50', 'g', 'kasztanów', 'jadalnych', 'z', 'puszki', 'w', 'wodzie,', 'pokrojonych', 'w', 'plastry', '(opcjonalnie)']
[]
['•', '2', 'łyżki', 'stołowe', 'sosu', 'do', 'pierożków', 'Lee', 'Kum', 'Kee']
['120g', 'makaronu', 'Mie', 'lub', 'z', 'warzywami']
['200g', 'filetu', 'z', 'kurczaka']
['1', 'łyżka', 'mąki', 'p

['2', 'łyżki', 'sezamu']
['2', 'łyżki', 'pasty', 'miso']
['2', 'łyżki', 'sosu', 'sojowego']
['1', 'limonka']
['1/2', 'łyżeczki', 'cukru']
['1', 'łyżka', 'oleju', 'sezamowego']
['4', 'łyżki', 'posiekanego', 'szczypiorku']
['świeża', 'kolendra']
['800g', 'filetu', 'z', 'łososia']
['1', 'łyżka', 'oliwy']
['Marynata:']
['6', 'łyżek', 'sosu', 'sojowego']
['8', 'łyżek', 'sosu', 'teriyaki']
['2', 'łyżki', 'miodu', 'lipowego']
['1', 'czubata', 'łyżeczka', 'imbiru', '(mielonego', 'lub', 'świeżo', 'startego', '-', '1,5cm)']
['2', 'duże', 'żabki', 'czosnku', '(przeciśnięte', 'przez', 'praskę', 'i', 'roztarte', 'z', 'solą)']
['sól', 'i', 'pieprz']
['Tempura', 'Samlip']
['olej', 'kokosowy']
['surowe', 'krewetki', '24', 'szt.']
['mąka', 'pszenna', '1', 'szklanka']
['sól']
['woda']
['Bulion:', '•', '1kg', 'kości', 'wieprzowych']
['1kg', 'kości', 'drobiowych']
['4', 'cebulki', 'szalotki']
['10cm', 'korzenia', 'imbiru']
['pół', 'główki', 'czosnku']
['2', 'marchewki']
['albi', 'Konbu', '(płat)']
['100ml

['2', 'łyżki', 'stołowe', 'oleju', 'roślinnego']
['1', 'łyżka', 'stołowa', 'sosu', 'rybnego']
['1', 'filiżanka', 'mleczka', 'kokosowego']
['1/4', 'filiżanki', 'liści', 'słodkiej', 'bazylii']
['8', 'liści', 'limonki', 'kaffir,', 'drobno', 'posiekanych']
['1/3', 'łyżki', 'stołowej', 'jasnego', 'sosu', 'sojowego']
['2', 'łyżki', 'stołowe', 'pasty', 'curry', 'panang']
['15', 'ml', 'oleju', 'do', 'woka', 'Thai', 'Pride']
['40', 'g', 'żółtej', 'pasty', 'curry', 'Thai', 'Pride']
['40', 'ml', 'wody']
['60', 'g', 'ziemniaków']
['150', 'g', 'mięsa', 'drobiowego']
['60', 'g', 'cebuli']
['160', 'ml', 'mleczka', 'kokosowego', 'Diamond']
['20', 'g', 'liści', 'bazylii']
['5', 'g', 'czerwonej', 'papryczki', 'chili']
['1', 'pierś', 'z', 'kurczaa']
['3', 'liście', 'limonki', 'kafir']
['1-2', 'łyżki', 'pasty', 'Panang']
['1', 'łyżeczka', 'sosu', 'rybnego']
['1', 'łyżeczka', 'cukru', 'trzcinowego', 'lub', 'palmowego']
['1', 'łyżka', 'oleju']
['100', 'ml', 'mleczka', 'kokosowego']
['2', 'liście', 'limonki'

['30', 'ml', 'soku', 'z', 'limonki']
['1/2', 'szklanki', 'nasion', 'granatu']
['2', 'szt.', 'zielonej', 'części', 'dymki']
['sól']
['pieprz']
['listki', 'mięty', 'do', 'dekoracji']
['jajka', '5', 'sztuk']
['soczewica', '100', 'g']
['cebula', 'czerwona', '50', 'g']
['papryczka', 'jalapeno', 'marynowana', '50', 'g']
['majonez', '1⁄2', 'szklanki']
['musztarda', 'Dijon', '10', 'g']
['sól']
['pieprz']
['natka', 'pietruszki']
['500', 'g', 'ciecierzycy']
['1', 'czerwona', 'papryka']
['4', 'łyżki', 'oliwy', 'z', 'oliwek']
['1', 'łyżka', 'soku', 'z', 'cytryny']
['1/2', 'łyżeczki', 'cukru']
['3', 'łyżeczki', 'kminu', 'rzymskiego']
['1', 'łyżeczka', 'płatków', 'chili']
['mała', 'szczypta', 'soli']
['1/2', 'łyżeczki', 'mielonego', 'pieprzu']
['1', 'ząbek', 'czosnku']
['1/3', 'młodej', 'cebuli']
['1', 'pomidor']
['3', 'łyżki', 'posiekanego', 'koperku']
['4', 'łyżki', 'posiekanej', 'natki', 'pietruszki']
['2', 'łyżki', 'posiekanego', 'szczypiorku']
['kilka', 'listków', 'mięty']
['2', 'łyżeczki', 'ol

['1', 'szklanka', 'fasoli', 'białej', '(fasolę', 'wcześniej', 'namoczyć)']
['1/2', 'pora']
['1', 'łyżeczkę', 'mielonego', 'kminu']
['skórka', 'z', 'połowy', 'cytryny']
['olej']
['4', 'laski', 'selera']
['2', 'ziemniaki']
['2', 'łyżki', 'pasty', 'Harissa']
['2', 'liście', 'laurowe']
['1,5-2', 'l', 'bulionu']
['sok', 'z', 'połowy', 'cytryny']
['pęczek', 'kolendry']
['sól']
['pieprz', 'czarny']
['2', 'cukinie']
['1', 'cebula']
['140', 'g', 'pomidorów', 'suszonych', 'w', 'zalewie']
['2', 'ząbki', 'czosnku']
['150', 'g', 'świeżego', 'szpinaku']
['200', 'g', 'koziego', 'sera', '(rolada)']
['450', 'g', 'pomidorów', 'z', 'puszki']
['szczypta', 'cynamonu']
['1', 'łyżeczka', 'suszonego', 'oregano']
['1/2', 'łyżeczki', 'wędzonej', 'słodkiej', 'papryki']
['brązowy', 'cukier']
['miód']
['syrop', 'z', 'agawy', '(opcjonalnie)']
['2', 'łyżki', 'harissy']
['sól']
['pieprz']
['oliwa', 'z', 'oliwek']
['250g', 'mielonej', 'wieprzowiny']
['¼', 'łyżeczki', 'sody', 'oczyszczonej']
['2', 'łyżki', 'pasty', 'ha

['1', 'łyżka', 'jasnego', 'sosu', 'sojowego']
['½', 'łyżki', 'sosu', 'rybnego']
['1', '½', 'łyżki', 'cukru']
['2', 'łyżki', 'prażonych', 'nasion', 'białego', 'sezamu']
['1', 'szklanka', 'ciepłego', 'mleka']
['2', 'szklanki', 'mąki']
['2', 'łyżki', 'cukru']
['2', 'łyżeczki', 'suszonych', 'drożdży']
['szczypta', 'soli']
['1', 'łyżka', 'oleju']
['brązowy', 'cukier']
['parę', 'łyżek', 'posiekanych', 'orzeszków', '(włoskich,', 'laskowych', 'lub', 'ziemnych)']
['1', 'łyżeczka', 'cynamonu']
['1', 'opakowanie', 'zupki', 'Chapaghetti']
['1', 'opakowanie', 'zupki', 'Neoguri', '(ostra', '/', 'łagodna)']
['6', 'szklanek', 'wody', '(ok.', '1,1l)']
['340', 'g', 'tofu']
['200', 'g', 'gorzkiej', 'czekolady']
['1/2', 'szklanki', 'wody']
['1/2', 'szklanki', 'cukru']
['dodatki', 'smakowe', 'np.', '1', 'łyżeczka', 'ekstraktu', 'z', 'wanilii,', '1', 'łyżka', 'likieru', 'pomarańczowego,', '1/3', 'łyżeczki', 'cynamonu,', 'szczypta', 'ostrej', 'papryki']
['45', 'ml', 'Mekhong', 'Thai', 'Spirit']
['25', 'ml', 

# Proof of concept

In [693]:
%run words.py

In [694]:
nie_jednostki = [
    'jogurtu',
    'banany',
    'ostrej',
    'czerwonej',
    'piersi',
    'szparagów',
    'kapustka',
    'cukier',
    'świeżego',
    'dojrzałe',
    'Rumu',
    'dojrzałe',
    'czystej',
    'smakowe',
    'tequili',
    'limonki',
    'fasoli',
    'żeberek',
    "truskawek",
    'mąki',
    'pokrojonego',
    'cienkie',
    'cebuli',
    'kukurydzy',
    'bambusa',
    'posiekanych',
    'oleju',
    'oliwek',
    'papryczka',
    'szczypiorek',
    'kiełków',
    'orzeszków',
    'trawy',
    'ziemniaków',
    'pomidorów',
    'moreli',
    'mleczka',
    'wódki',
    'koktajlowe'
]

In [695]:
jednostki = Sentence(" ".join(jednostki))
nie_jednostki = Sentence(" ".join(nie_jednostki))
emb.embed(jednostki)
emb.embed(nie_jednostki)

[Sentence: "jogurtu banany ostrej czerwonej piersi szparagów kapustka cukier świeżego dojrzałe Rumu dojrzałe czystej smakowe tequili limonki fasoli żeberek truskawek mąki pokrojonego cienkie cebuli kukurydzy bambusa posiekanych oleju oliwek papryczka szczypiorek kiełków orzeszków trawy ziemniaków pomidorów moreli mleczka wódki koktajlowe" - 39 Tokens]

In [696]:
X_jed = torch.cat([j.embedding for j in jednostki]).reshape(len(jednostki),300)
X_njed = torch.cat([j.embedding for j in nie_jednostki]).reshape(len(nie_jednostki),300)

In [697]:
def knn(Y):
    mask = Y.norm(dim=1) != 0
    if not mask.any(): return torch.BoolTensor(mask)
    
    cos_sim_jed = X_jed @ Y[mask,:].T / (X_jed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)
    cos_sim_njed = X_njed @ Y[mask,:].T / (X_njed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)

    ret = torch.zeros(Y.shape[0], dtype=torch.bool).to(device)
    ret[mask] = (cos_sim_jed.max(0)[0] > cos_sim_njed.max(0)[0]) & (cos_sim_jed.max(0)[0] > 0.5)
    return ret

In [698]:
def sentenceToMat(s):
    s = Sentence(s)
    emb.embed(s)
    r = torch.cat([i.embedding for i in s])
    return r.reshape(int(r.shape[0]/300), 300), s

In [699]:
knn(sentenceToMat('1 łyżka bezalkoholowy z guawą i imbirem mega')[0])

tensor([False,  True, False, False, False, False, False, False])

In [700]:
knn(sentenceToMat('234 Łyżeczek kefiru')[0])

tensor([False,  True, False])

In [701]:
knn(sentenceToMat('rabarbar 4 łodygi')[0])

tensor([False, False,  True])

In [702]:
Unit = []
for d in data:
    rec = []
    for i in d['ingredients']:
        mat, s = sentenceToMat(i[0])
        mask = knn(mat)
        rec.append([s[i] for i in range(len(s)) if mask[i]])
    Unit.append(rec)

In [703]:
Units_per_recipe = [[" ".join([u.text for u in units]) for units in U] for U in Unit] 

In [704]:
Unit

[[[Token: 2 łyżeczki, Token: 3 soku],
  [Token: 2 szklanki],
  [Token: 2 łyżki, Token: 3 miodu],
  [Token: 2 łyżeczki]],
 [[], [Token: 2 owoców], [], [], [Token: 2 szklanka], []],
 [[Token: 2 szklanka],
  [Token: 2 szklanka, Token: 3 owoców],
  [],
  [],
  [Token: 2 łyżka]],
 [[Token: 2 szklanki],
  [Token: 2 szklanki],
  [Token: 2 łyżeczki],
  [Token: 2 szklanka],
  [Token: 2 łyżki, Token: 4 nasion],
  [Token: 2 łyżeczki, Token: 3 soku]],
 [[], [], [], [], []],
 [[], [], [], [], [Token: 2 szklanka, Token: 3 soku, Token: 4 jabłkowego]],
 [[], [], [], [], [], [], [Token: 1 Dodatki], []],
 [[], [], [], [Token: 5 listków]],
 [[],
  [],
  [],
  [],
  [],
  [],
  [Token: 2 krewetek],
  [],
  [],
  [Token: 3 ząbek],
  [],
  [],
  [],
  []],
 [[],
  [Token: 2 łyżki],
  [],
  [Token: 2 liści],
  [],
  [],
  [Token: 2 łyżki],
  [Token: 2 łyżki],
  [Token: 2 łyżeczka],
  [Token: 4 soku]],
 [[Token: 2 g],
  [Token: 2 g],
  [Token: 2 g],
  [],
  [],
  [Token: 1 kawałek],
  [],
  [],
  [],
  [Token

# Full

In [705]:
przyprawy, warzywa, nabial, zboza, tluszcze, miesa;

NameError: name 'miesa' is not defined

In [706]:
def knn(Y, X_jed, X_njed, d):
    mask = Y.norm(dim=1) != 0
    if not mask.any(): return torch.BoolTensor(mask)
    
    cos_sim_jed = X_jed @ Y[mask,:].T / (X_jed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)
    cos_sim_njed = X_njed @ Y[mask,:].T / (X_njed.norm(dim=1, keepdim=True) * Y[mask,:].norm(dim=1, keepdim=True).T)

    ret = torch.zeros(Y.shape[0], dtype=torch.bool).to(device)
    
    ret[mask] = (d*cos_sim_jed.max(0)[0] > cos_sim_njed.max(0)[0]) & (cos_sim_jed.max(0)[0] > 0.5)
    return ret

In [707]:
def gen_knn(x, nx, d):
    x = Sentence(" ".join(x))
    nx = Sentence(" ".join(nx))
    emb.embed(x)
    emb.embed(nx)
    x = [xx for xx in x if xx.embedding.norm() > 0.001]
    nx = [xx for xx in nx if xx.embedding.norm() > 0.001]
    X_x = torch.cat([j.embedding for j in x]).reshape(len(x),300)
    X_nx = torch.cat([j.embedding for j in nx]).reshape(len(nx),300)
    return lambda Y: knn(Y, X_x, X_nx, d)

In [708]:
s = Sentence("lód asdf piwo")

In [709]:
emb.embed(s)

[Sentence: "lód asdf piwo" - 3 Tokens]

In [710]:
[sss.embedding for sss in [ss for ss in s if ss.embedding.norm() > 0.001]]

[tensor([-2.8112e-01, -5.0371e-02,  4.8964e-02, -6.7958e-01,  7.4694e-01,
          2.0344e-01,  1.2213e-01, -1.9825e-01,  1.3946e-01,  5.3816e-01,
          4.6904e-01,  9.8772e-02,  4.3799e-01,  2.9822e-01, -1.3622e-01,
          5.4849e-01,  2.3856e-01,  2.7953e-01,  5.4909e-02, -1.3054e-01,
         -3.9186e-01, -8.3339e-02, -6.2480e-01, -1.0207e-02, -4.2332e-01,
          2.4943e-01, -2.0429e-01, -9.1938e-02, -2.8870e-01, -2.6780e-02,
         -1.8569e-01, -1.5805e-01,  2.6072e-01, -2.9521e-01,  4.3091e-01,
          5.5229e-01,  2.2313e-02,  2.3725e-01, -1.7151e-01, -9.3433e-04,
         -3.3377e-01,  4.8355e-02, -1.1487e-01, -2.0072e-01,  2.6008e-01,
         -8.1873e-01,  1.1706e-01, -2.3152e-01, -1.1688e-01, -1.7142e-01,
          3.1835e-01, -8.2847e-02,  5.8493e-01,  3.5867e-01,  4.6979e-02,
         -1.8964e-01, -1.6925e-01, -6.0271e-01,  4.0612e-01,  1.6662e-01,
         -9.4596e-02, -2.0708e-01,  3.1231e-01, -1.7679e-01,  2.4070e-01,
          6.0691e-02,  6.7991e-01,  1.

In [711]:
def predict(knn, data):
    Unit = []
    for d in data:
        rec = []
        for i in d['ingredients']:
            mat, s = sentenceToMat(i[0])
            mask = knn(mat)
            rec.append([s[i] for i in range(len(s)) if mask[i]])
        Unit.append(rec)
        
    return [[" ".join([u.text for u in units]) for units in U] for U in Unit] 

# Jednostki

In [712]:
%run words.py

In [713]:
knn_jednostki = gen_knn(jednostki, przyprawy + warzywa + nabial + zboza + tluszcze, 1)

In [714]:
predict(knn_jednostki, data)

[['łyżeczki soku', 'szklanki', 'łyżki', 'łyżeczki'],
 ['', 'owoców', '', '', 'szklanka', ''],
 ['szklanka', 'szklanka owoców', '', '', 'łyżka'],
 ['szklanki',
  'szklanki mleczka',
  'łyżeczki',
  'szklanka',
  'łyżki',
  'łyżeczki soku'],
 ['', '', '', 'truskawek', ''],
 ['', '', '', '', 'szklanka soku jabłkowego'],
 ['', '', '', '', '', '', 'Dodatki', ''],
 ['', '', '', 'listków'],
 ['', '', '', '', '', '', 'krewetek', '', '', 'ząbek', '', '', '', ''],
 ['',
  'łyżki',
  '',
  'liści',
  'pokrojonej',
  '',
  'łyżki',
  'łyżki',
  'łyżeczka',
  'soku'],
 ['g', 'g', 'g', '', '', 'kawałek', '', '', '', 'ząbek', '', 'łyżka'],
 ['g', '', '', '', 'ząbki', '', '', '', '', '', 'litr', 'ml mleczka'],
 ['ml', 'g', 'ml', 'g', 'g', 'ml mleczka', 'g liści', 'g'],
 ['ćwiartki',
  'g',
  'g',
  'g',
  'g jagód',
  'łyżki',
  'łyżki',
  'łyżeczki',
  'łyżka',
  'łyżki',
  'ml'],
 ['szklanka',
  'ml',
  'cienkie',
  'łyżka',
  '',
  'łyżki',
  '',
  '',
  '',
  'garść',
  'łyżki',
  'łyżeczka',
  'ł

In [715]:
nie_przyprawy = ['stołowa']

In [716]:
knn_przyprawy = gen_knn(przyprawy, jednostki + warzywa + nabial + zboza + tluszcze + nie_przyprawy, 1)

In [717]:
predict(knn_przyprawy, data)

[['', '', '', 'imbiru'],
 ['', '', '', '', '', ''],
 ['', 'mango', '', '', 'cukru'],
 ['', '', '', '', '', ''],
 ['', '', 'mango', '', ''],
 ['Mango', 'cytryny', 'mandarynek', '', ''],
 ['Rumu', '', 'trawy', '', '', '', '', 'cytryna'],
 ['Mango', '', '', 'mango mięty'],
 ['', '', '', 'szyjkami', '', '', '', '', '', 'cały', '', '', '', ''],
 ['', '', '', '', '', '', '', '', 'imbiru', 'sól pieprz'],
 ['', '', '', '', '', '', 'ziela', '', '', '', '', ''],
 ['', '', '', '', '', 'kmin', '', '', '', '', '', ''],
 ['', 'curry', '', '', '', '', '', ''],
 ['', 'Rice', 'imbiru', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', 'pieprz', '', '', '', '', '', '', '', 'pieprz'],
 ['', '', '', '', '', '', 'cukru', 'cytryna'],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', 'cukru', 'mięty', 'cytryna', '', '', 'gorzkiej'],
 ['', '', '', '', '', 'mozzarelli', '', ''],
 ['', '', '', '', '', '', '', '', '', '', 'kolendry'],
 ['', '', '', '', '', '', '', '', '', '', ''],


In [718]:
nie_warzywa = ['zupy', 'sosów', 'małży', 'smażonej', 'marynowanych', 'musztardy', 'chleba', 'świeże', 'ugotowanych', 'czarny', 'japońskiej']

In [719]:
knn_warzywa = gen_knn(warzywa, jednostki + przyprawy + nabial + zboza + tluszcze + nie_warzywa, 1)

In [720]:
predict(knn_warzywa, data)

[['', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', 'natka'],
 ['', '', 'fasoli', '', '', '', '', '', '', ''],
 ['fasoli', 'żeberek', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', 'chińskiej', ''],
 ['', '', '', '', 'średnia', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', 'mięsnego', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['placki', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', '', 'sałata', '', '', ''],
 ['', '', '', '', '', '',

In [721]:
nie_nabial = ['Napój', 'cukier', 'gazowanego', 'w', 'pasty', 'groszku', 'bulionu', 'kleistego']

In [722]:
knn_nabial = gen_knn(nabial, jednostki + przyprawy + warzywa + zboza + tluszcze + nie_nabial, 1)

In [723]:
predict(knn_nabial, data)

[['', '', '', ''],
 ['jogurtu', '', '', '', '', ''],
 ['', '', '', 'jogurtu', ''],
 ['', '', '', '', '', ''],
 ['', 'jogurtu', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', 'soli', '', 'wódki wódki', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', 'mleka', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', 'sproszkowanego', '', '', '', 'octu', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', 'sody proszku', 'soli', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', 

In [724]:
knn_zboza = gen_knn(zboza, jednostki + przyprawy + warzywa + nabial + tluszcze, 1)

In [725]:
predict(knn_zboza, data)

[['', '', '', ''],
 ['', '', '', '', '', 'miód'],
 ['', '', '', '', 'brązowego'],
 ['puree', '', 'brązowego', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', 'syropu', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', 'białego', '', '', ''],
 ['makaronu', '', '', '', '', 'smażonego Tofu', '', 'sosu', '', ''],
 ['', '', 'wędzonego', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', 'polędwicy', '', '', ''],
 ['kurczaka', '', '', '', '', '', 'sosu', '', 'smażenia', 'ryżowej', ''],
 ['',
  'bulionu drobiowego',
  'obrane pokrojone',
  '',
  'pokrojona drobną',
  'mąki',
  '',
  '',
  '',
  '',
  'parmezanu',
  '',
  '',
  '',
  '',
  ''],
 ['białe', 'kurczaka', 'makaronu', 'bulionu', '', 'sosu', '', ''],
 ['', '', '', 'sosu', '', 'mąki', '', ''],
 ['', '', 'prażonej', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['tortilli', '', '', 'posiekanego', '', 'sera', '', ''],
 ['', '', '',

In [726]:
knn_tluszcze = gen_knn(tluszcze, zboza + jednostki + przyprawy + warzywa + nabial, 0.8)

In [727]:
predict(knn_tluszcze, data)

[['', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', 'oliwy', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', 'oliwa', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['',
  '',
  '',
  'oliwy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'orzeszków pestek',
  'oliwy',
  '',
  '',
  ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', 'oliwa', '', '', '', '', ''],
 ['orzechów', '', '', 'himalajskiej', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '',

In [728]:
knn_skladnik = gen_knn(tluszcze + zboza + przyprawy + warzywa + nabial, jednostki, 1)

In [729]:
predict(knn_skladnik, data)

[['limonki', 'napoju', 'miodu', 'imbiru'],
 ['jogurtu', '', '', '', '', 'miód'],
 ['', 'mango', '', 'jogurtu', 'cukru brązowego'],
 ['puree', '', 'brązowego', '', '', 'limonki'],
 ['kokosowej', 'jogurtu', 'mango', '', ''],
 ['Napój Mango', 'cytryny', 'mandarynek', 'ananasa', ''],
 ['Rumu',
  'Wina Sake Gekkeikan',
  'syropu trawy',
  '',
  'kostek',
  '',
  '',
  'cytryna'],
 ['Napój Mango', 'herbaty', 'kostek', 'mango mięty'],
 ['zupy',
  'pasty sosów',
  'zupy',
  'małży szyjkami',
  'małży',
  '',
  '',
  'papryczki chili',
  '',
  'cały',
  'białego',
  'oliwy',
  '',
  'natka pietruszki'],
 ['makaronu',
  'oleju',
  'fasoli',
  '',
  'papryki',
  'smażonego Tofu',
  '',
  'sosu sojowego',
  'imbiru',
  'sól pieprz'],
 ['fasoli',
  'żeberek',
  'wędzonego',
  '',
  '',
  '',
  'ziela',
  '',
  '',
  '',
  '',
  'majeranku'],
 ['', '', '', '', '', 'kmin', '', '', '', 'oliwa', '', 'kokosowego'],
 ['oleju',
  'pasty curry',
  '',
  '',
  'polędwicy',
  'kokosowego',
  '',
  'papryczki

In [730]:
[d['ingredients'] for d in data]

[[['3 łyżeczki soku z limonki \r'],
  ['1/2 szklanki napoju Rubicon Deluxe Guawa \r'],
  ['2 łyżki miodu \r'],
  ['2 łyżeczki imbiru ']],
 [['300ml jogurtu naturalnego 2%\r'],
  ['10 owoców lychee\r'],
  ['2 banany\r'],
  ['200g ananasa\r'],
  ['1 szklanka kruszonego lodu\r'],
  ['miód']],
 [['1 szklanka wody kokosowej\r'],
  ['1 szklanka owoców mango (świeżych lub mrożonych)\r'],
  ['1 obrany banan\r'],
  ['200g jogurtu greckiego\r'],
  ['1 łyżka cukru brązowego']],
 [['2 szklanki puree z mango\r'],
  ['1/2 szklanki mleczka kokosowego\r'],
  ['2 łyżeczki brązowego cukru\r'],
  ['1 szklanka wody mineralnej niegazowanej\r'],
  ['4 łyżki stołowe nasion Chia\r'],
  ['2 łyżeczki soku z limonki']],
 [['100ml wody kokosowej \r'],
  ['100g jogurtu naturalnego \r'],
  ['1/2 mango \r'],
  ['150g truskawek \r'],
  ['lód ']],
 [['Napój Mango Rubicon Deluxe \r'],
  ['1/2 cytryny \r'],
  ['6 mandarynek \r'],
  ['125g ananasa \r'],
  ['1 szklanka soku jabłkowego ']],
 [['30ml Rumu Tanduay\r'],
  ['6